In [1]:
import numpy as np
import tinygrad 
from tinygrad.extra.datasets import fetch_cifar
import time
from tinygrad.lazy import Device
Device.DEFAULT = "CUDA"

import cv2
start =time.monotonic()
X_train, Y_train,= fetch_cifar(train=True)

In [2]:
from tinygrad.nn import Linear, Conv2d
from tinygrad.nn import optim
from tinygrad.tensor import Tensor
Tensor.training= True

class TinyCIFAR:
    def __init__ (self):
        self.l1 = Linear(3072,15360, bias=True)
        self.l2 = Linear(15360,255,bias=True)
        self.l3 = Linear(255,10,bias=True)
    def __call__ (self, x):
        x = self.l1(x)
        x = x.leakyrelu()
        x = self.l2(x)
        x = x.leakyrelu()
        x = self.l3(x)
        return x.log_softmax()
    #here lies the buggers
    #maybe a dense just doesnt learn CIFAR? 
    
net = TinyCIFAR()

In [3]:
print(X_train.shape)
X_trains = X_train.reshape(50000, -1)
print(X_trains.shape)
from tinygrad.nn.optim import SGD, Adam
opt = SGD([net.l1.weight, net.l2.weight, net.l3.weight], lr=3e-3)

def cross_entropy(out, Y):
  num_classes = out.shape[-1]
  YY = Y.flatten().astype(np.int32)
  y = np.zeros((YY.shape[0], num_classes), np.float32)
  y[range(y.shape[0]),YY] = -1.0*num_classes
  y = y.reshape(list(Y.shape)+[num_classes])
  y = Tensor(y)
  return out.mul(y).mean()

# print(Y_train.shape)
# print(X_trains[5,:].max())
# X_trains = np.divide(X_trains, 256)
# print(X_trains[:,:].max())


(50000, 3, 32, 32)
(50000, 3072)


In [4]:




BS=32

#Train the modelieren

for step in range(12000):
    samp=np.random.randint(0, X_trains.shape[0], BS)
    batch = Tensor(X_trains[0], requires_grad=True)
    labels = Y_train[samp]
    out = net(batch)
    loss = cross_entropy(out, labels)
    #print(loss.numpy())
    opt.zero_grad() #NOTE:I'm unsure why it's giving me t.grad is not None (it was array shaping)
    loss.backward()
    opt.step()
    #accuracy dealings
    pred = np.argmax(out.numpy(), axis=-1)
    acc = (pred == labels).mean()
    # if acc >= 1: #how optimistic of myself
    #     break
    if step % 25 == 0:
        print(f'step{step} | loss: {loss.numpy()} | Accuracy: {acc}')
    if step == 2000: 
        if acc < 0.25:
            print('Training but not learning')
            break


end=time.monotonic()
print(f'{end-start:.3}s')



step0 | loss: 53.54902267456055 | Accuracy: 0.125
step25 | loss: nan | Accuracy: 0.09375
step50 | loss: nan | Accuracy: 0.125
step75 | loss: nan | Accuracy: 0.0625
step100 | loss: nan | Accuracy: 0.0625
step125 | loss: nan | Accuracy: 0.125
step150 | loss: nan | Accuracy: 0.03125
step175 | loss: nan | Accuracy: 0.125
step200 | loss: nan | Accuracy: 0.09375
step225 | loss: nan | Accuracy: 0.09375
step250 | loss: nan | Accuracy: 0.03125
step275 | loss: nan | Accuracy: 0.15625
step300 | loss: nan | Accuracy: 0.09375
step325 | loss: nan | Accuracy: 0.03125
step350 | loss: nan | Accuracy: 0.1875
step375 | loss: nan | Accuracy: 0.1875
step400 | loss: nan | Accuracy: 0.125
step425 | loss: nan | Accuracy: 0.03125
step450 | loss: nan | Accuracy: 0.15625
step475 | loss: nan | Accuracy: 0.03125
step500 | loss: nan | Accuracy: 0.09375
step525 | loss: nan | Accuracy: 0.09375
step550 | loss: nan | Accuracy: 0.125
step575 | loss: nan | Accuracy: 0.15625
step600 | loss: nan | Accuracy: 0.21875
step625

In [5]:
#Validate model
X_test, Y_test= fetch_cifar(train=False)
print(X_test.shape, Y_test.shape)
av_acc = 0 #loopy fucks up without this
X_tests = X_test.reshape(10000, -1)
print(X_tests.shape)
testamount = 100
for step in range(testamount):
    samp = np.random.randint(0, X_tests.shape[0], size=BS)
    batch = Tensor(X_tests[samp], requires_grad=True)
    labels = Y_test[samp]
    out = net (batch)

    pred = np.argmax(out.numpy(), axis=-1)
    av_acc += (pred == labels).mean()
    # print(out.numpy())
    # #print(np.argmax(out.numpy()))
    # print(Y_test[samp])
    # #print(batch.numpy())

print(f"Test Accuracy: {av_acc / testamount}")

(10000, 3, 32, 32) (10000,)
(10000, 3072)
Test Accuracy: 0.09125
